In [0]:
# Import libraries
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path
import tensorflow as tf
from typing import List, Union
import warnings

# Below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

# Imports for accessing Datathon data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

# Install required packages
# !pip install -q tsfresh
# !pip install -q dask
import tsfresh
import dask
import dask.dataframe as dd

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [0]:
os.getcwd()

'/content'

# Run queries

In [0]:
auth.authenticate_user()
project_id='milan-datathon'
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id

In [0]:
# Read data from BigQuery into pandas dataframes.
def run_query(query):
  return pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, configuration={'query':{'useLegacySql': False}})

## Get pairs of blood pressure values

In [0]:
max_minutes = 3

df = run_query(
f'''
WITH invasive_bpa AS (SELECT *
FROM `physionet-data.mimiciii_clinical.chartevents` ce
WHERE itemid = 220052
ORDER BY charttime
 ),
non_inv_bpa as (SELECT *
FROM `physionet-data.mimiciii_clinical.chartevents` ce
WHERE itemid = 220181
ORDER BY charttime
),
temp as (SELECT 
  invasive_bpa.icustay_id as id,
  invasive_bpa.value as invasive_value, 
  non_inv_bpa.value as ninv_value, 
  invasive_bpa.charttime as invasive_time,
  non_inv_bpa.charttime as ninv_time,
  ABS(DATETIME_DIFF(invasive_bpa.charttime, non_inv_bpa.charttime, MINUTE)) as time_diff
from invasive_bpa 
join non_inv_bpa
ON invasive_bpa.icustay_id = non_inv_bpa.icustay_id
WHERE ABS(DATETIME_DIFF(invasive_bpa.charttime, non_inv_bpa.charttime, MINUTE)) <= {max_minutes}
),
grouped_temp as (SELECT
  temp.id as id,
  temp.time_diff as time_diff,
  temp.invasive_value as invasive_value, 
  temp.ninv_value as ninv_value,
  temp.invasive_time as invasive_time,
  temp.ninv_time as ninv_time,
  RANK() OVER (PARTITION BY temp.id, temp.invasive_time ORDER BY temp.time_diff ASC) as ranky
FROM temp
)

SELECT * FROM grouped_temp
WHERE ranky = 1
'''
)

## Get femoral and radial lines

Get the presence of invasive lines

In [0]:
df_femoral = run_query(
'''
SELECT ICUSTAY_ID, STARTTIME, ENDTIME, VALUE as LENGTH_MINUTES FROM `physionet-data.mimiciii_clinical.procedureevents_mv`
WHERE ORDERCATEGORYNAME = 'Invasive Lines'
AND LOCATION IN ('Left Femoral', 'Left Femoral.','Right Femoral', 'Right Femoral.')
ORDER BY ICUSTAY_ID, STARTTIME
'''
)
 
df_radial = run_query(
'''
SELECT ICUSTAY_ID, STARTTIME, ENDTIME, VALUE as LENGTH_MINUTES FROM `physionet-data.mimiciii_clinical.procedureevents_mv`
WHERE ORDERCATEGORYNAME = 'Invasive Lines'
AND LOCATION IN ('Left Radial','Right Radial', 'Left Radial.', 'Right Radial.')
ORDER BY ICUSTAY_ID, STARTTIME
'''
)

### Process BP pairs based on invasive lines
Create labels for each BP to check whether the invasive measurement is femoral or radial.
Drop those that we are unsure of (e.g. that have no or both lines)

In [0]:
df_bp_merged = df.merge(df_femoral, how='left', left_on='id', right_on='ICUSTAY_ID')
df_bp_merged['femoral_yn'] = 0
df_bp_merged.loc[df_bp_merged['invasive_time'].between(df_bp_merged['STARTTIME'], df_bp_merged['ENDTIME']), 'femoral_yn'] = 1
df_bp_merged = df_bp_merged.drop(['ICUSTAY_ID', 'STARTTIME', 'ENDTIME', 'LENGTH_MINUTES'], axis=1)
df_bp_merged = df_bp_merged.merge(df_radial, how='left', left_on='id', right_on='ICUSTAY_ID')
df_bp_merged['radial_yn'] = 0
df_bp_merged.loc[df_bp_merged['invasive_time'].between(df_bp_merged['STARTTIME'], df_bp_merged['ENDTIME']), 'radial_yn'] = 1
df_bp_merged = df_bp_merged.drop(['ICUSTAY_ID', 'STARTTIME', 'ENDTIME', 'LENGTH_MINUTES','ranky'], axis=1)

In [0]:
df_bp_final = (df_bp_merged.drop(df_bp_merged[(df_bp_merged['femoral_yn'] == 1) & (df_bp_merged['radial_yn']==1)].index)
                          .drop(df_bp_merged[(df_bp_merged['femoral_yn'] == 0) & (df_bp_merged['radial_yn']==0)].index))

In [0]:
df_bp_final = df_bp_final.groupby(by='id', as_index=False).first()

(4923, 8)


,id,time_diff,invasive_value,ninv_value,invasive_time,ninv_time,femoral_yn,radial_yn
0,200033,0,54,59,2198-08-10 19:00:00,2198-08-10 19:00:00,0,1
1,200063,0,68,59,2141-03-13 09:00:00,2141-03-13 09:00:00,0,1
2,200069,0,78,61,2195-12-12 10:00:00,2195-12-12 10:00:00,0,1
3,200075,0,77,54,2159-09-23 03:30:00,2159-09-23 03:30:00,0,1
4,200141,3,56,51,2143-10-15 14:50:00,2143-10-15 14:53:00,0,1


## Get features

Get and process predictive features (lab results, meds, etc)

### Patient time-independent features

In [0]:
df_features = run_query(
   '''
    select * from `milan-datathon.temp7.patient_info_on_icustay`
   '''
)

### Time-dependent features

#### Vasopressin

#### Query to get last measurements before BP pairing

Each signal query will be joined to this one

In [0]:
query_all = """invasive_bpa AS (SELECT *
FROM `physionet-data.mimiciii_clinical.chartevents` ce
WHERE itemid = 220052
ORDER BY charttime
 ),
non_inv_bpa as (SELECT *
FROM `physionet-data.mimiciii_clinical.chartevents` ce
WHERE itemid = 220181
ORDER BY charttime
),
temp as (SELECT 
  invasive_bpa.icustay_id as id,
  invasive_bpa.value as invasive_value, 
  non_inv_bpa.value as ninv_value, 
  invasive_bpa.charttime as invasive_time,
  non_inv_bpa.charttime as ninv_time,
  ABS(DATETIME_DIFF(invasive_bpa.charttime, non_inv_bpa.charttime, MINUTE)) as time_diff
from invasive_bpa 
join non_inv_bpa
ON invasive_bpa.icustay_id = non_inv_bpa.icustay_id
WHERE ABS(DATETIME_DIFF(invasive_bpa.charttime, non_inv_bpa.charttime, MINUTE)) <= 3
),
grouped_temp as (SELECT
  temp.id as id,
  temp.time_diff as time_diff,
  temp.invasive_value as invasive_value, 
  temp.ninv_value as ninv_value,
  temp.invasive_time as invasive_time,
  temp.ninv_time as ninv_time,
  RANK() OVER (PARTITION BY temp.id, temp.invasive_time ORDER BY temp.time_diff ASC) as ranky
FROM temp
),

final_bp as (SELECT * FROM grouped_temp
WHERE ranky = 1
),

ranked as (select 
    hr.icustay_id, MAX(hr.CHARTTIME) as time
    from hr
    left join final_bp on final_bp.id = hr.icustay_id
    where hr.CHARTTIME <= final_bp.invasive_time
    group by hr.icustay_id
)

SELECT ranked.icustay_id, hr.value from ranked
left join hr on ranked.time = hr.charttime and ranked.icustay_id = hr.icustay_id"""

#### Heart rate

In [0]:
df_heartsingle = run_query(
f"""
WITH hr as (SELECT
  a.ICUSTAY_ID,
  a.CHARTTIME,
  a.STORETIME,
  a.CGID,
  a.VALUE,
  a.VALUENUM,
  a.VALUEUOM,
  a.WARNING,
  a.ERROR,
  a.RESULTSTATUS,
  a.STOPPED,
  b.label,
  a.itemid
FROM `physionet-data.mimiciii_clinical.chartevents` as a
INNER JOIN `physionet-data.mimiciii_clinical.d_items` as b
ON a.itemid = b.itemid
WHERE upper(b.label) LIKE '%HEART%'
AND upper(valueuom) LIKE 'BPM'),

{query_all}
""")

In [0]:
df_heartsingle['value'] = pd.to_numeric(df_heartsingle['value'], errors='coerce')
df_heartsingle = df_heartsingle.groupby(by='icustay_id',as_index=False)['value'].mean()
df_heartsingle.head(n=5)

#### Saturation

In [0]:
df_sat = run_query(
f"""
WITH HR as (SELECT 
  ce.ICUSTAY_ID,
  ce.ITEMID,
  ce.CHARTTIME,
  ce.STORETIME,
  ce.CGID,
  ce.VALUE,
  ce.VALUENUM,
  ce.VALUEUOM,
  ce.WARNING,
  ce.ERROR,
  ce.RESULTSTATUS,
  ce.STOPPED
FROM `physionet-data.mimiciii_clinical.chartevents` as ce
INNER JOIN `milan-datathon.temp7.icustays` as icustays
ON icustays.ids = ce.icustay_id
WHERE ITEMID in (646, 220277) -- CV and MV
ORDER BY icustay_id
),

{query_all}
""")

In [0]:
df_sat['value'] = pd.to_numeric(df_sat['value'], errors='coerce')
df_sat = df_sat.groupby(by='icustay_id',as_index=False)['value'].mean()


#### Vasopressors

In [0]:
df_vasopressor = run_query(
f"""WITH HR as (

SELECT
  dose.icustay_id,
  dose.starttime as CHARTTIME,
  dose.endtime,
  dose.vaso_rate,
  dose.vaso_amount as VALUE       
FROM `physionet-data.mimiciii_derived.vasopressin_dose` as dose
INNER JOIN `milan-datathon.temp7.icustays` as icustays
ON icustays.ids = dose.icustay_id
ORDER BY icustay_id)
,

{query_all}
""")

In [0]:
df_vasopressor['value'] = pd.to_numeric(df_vasopressor['value'], errors='coerce')
df_vasopressor = df_vasopressor.groupby(by='icustay_id',as_index=False)['value'].mean()

#### Diagnoses

In [0]:
df_diag = run_query(
f"""
SELECT
 icustay.icustay_id,
 diag.icd9_code,
 diag_types.short_title,
 diag_types.long_title
FROM `physionet-data.mimiciii_clinical.icustays` AS icustay
INNER JOIN `physionet-data.mimiciii_clinical.diagnoses_icd` AS diag
ON icustay.hadm_id = diag.hadm_id
INNER JOIN `physionet-data.mimiciii_clinical.d_icd_diagnoses` as diag_types
ON diag.icd9_code = diag_types.icd9_code
INNER JOIN `milan-datathon.temp7.icustays` as icustays
ON icustays.ids = icustay.icustay_id
""")

In [0]:
# Only keep most common diagnoses
vc = df_diag['short_title'].value_counts()[:20].index.tolist()
df_diag['diagnosis_cat'] = df_diag['short_title'].apply(lambda x: x if x in vc else 'Other')

In [0]:
for col in df_diag['diagnosis_cat'].unique():
  df_bp_merged[col] = 0
  patients_with_diag = df_diag.loc[df_diag['diagnosis_cat'] == col, 'icustay_id'].unique().tolist()
  df_bp_merged.loc[df_bp_merged['id'].isin(patients_with_diag), col] = 1

#### Lab results

In [0]:
df_lab_ph = run_query(f""" WITH HR as (
SELECT
  a.itemid,
  b.label,
  a.HADM_ID,
  c.icustay_id,
  a.CHARTTIME,
  a.VALUE,
  a.VALUENUM,
  a.VALUEUOM,
  a.FLAG
FROM `physionet-data.mimiciii_clinical.labevents` AS a
INNER JOIN `physionet-data.mimiciii_clinical.d_labitems` AS b
ON a.itemid = b.itemid
INNER JOIN `physionet-data.mimiciii_clinical.icustays` as c
ON c.hadm_id = a.hadm_id
WHERE a.itemid in (50820)
AND a.hadm_id IS NOT NULL)
,

{query_all}
""")

In [0]:
df_lab_hemaglobin = run_query(f""" WITH HR as (
SELECT
  a.itemid,
  b.label,
  a.HADM_ID,
  c.icustay_id,
  a.CHARTTIME,
  a.VALUE,
  a.VALUENUM,
  a.VALUEUOM,
  a.FLAG
FROM `physionet-data.mimiciii_clinical.labevents` AS a
INNER JOIN `physionet-data.mimiciii_clinical.d_labitems` AS b
ON a.itemid = b.itemid
INNER JOIN `physionet-data.mimiciii_clinical.icustays` as c
ON c.hadm_id = a.hadm_id
WHERE a.itemid in (50811)
AND a.hadm_id IS NOT NULL)
,

{query_all}
""")

In [0]:
df_lab_crp = run_query(f""" WITH HR as (
SELECT
  a.itemid,
  b.label,
  a.HADM_ID,
  c.icustay_id,
  a.CHARTTIME,
  a.VALUE,
  a.VALUENUM,
  a.VALUEUOM,
  a.FLAG
FROM `physionet-data.mimiciii_clinical.labevents` AS a
INNER JOIN `physionet-data.mimiciii_clinical.d_labitems` AS b
ON a.itemid = b.itemid
INNER JOIN `physionet-data.mimiciii_clinical.icustays` as c
ON c.hadm_id = a.hadm_id
WHERE a.itemid in (50889)
AND a.hadm_id IS NOT NULL)
,

{query_all}
""")

In [0]:
df_lab_po2 = run_query(f""" WITH HR as (
SELECT
  a.itemid,
  b.label,
  a.HADM_ID,
  c.icustay_id,
  a.CHARTTIME,
  a.VALUE,
  a.VALUENUM,
  a.VALUEUOM,
  a.FLAG
FROM `physionet-data.mimiciii_clinical.labevents` AS a
INNER JOIN `physionet-data.mimiciii_clinical.d_labitems` AS b
ON a.itemid = b.itemid
INNER JOIN `physionet-data.mimiciii_clinical.icustays` as c
ON c.hadm_id = a.hadm_id
WHERE a.itemid in (50821)
AND a.hadm_id IS NOT NULL)
,

{query_all}
""")

In [0]:
df_lab_base_excess = run_query(f""" WITH HR as (
SELECT
  a.itemid,
  b.label,
  a.HADM_ID,
  c.icustay_id,
  a.CHARTTIME,
  a.VALUE,
  a.VALUENUM,
  a.VALUEUOM,
  a.FLAG
FROM `physionet-data.mimiciii_clinical.labevents` AS a
INNER JOIN `physionet-data.mimiciii_clinical.d_labitems` AS b
ON a.itemid = b.itemid
INNER JOIN `physionet-data.mimiciii_clinical.icustays` as c
ON c.hadm_id = a.hadm_id
WHERE a.itemid in (50802)
AND a.hadm_id IS NOT NULL)
,

{query_all}
""")

In [0]:
df_lab_glucose = run_query(f""" WITH HR as (
SELECT
  a.itemid,
  b.label,
  a.HADM_ID,
  c.icustay_id,
  a.CHARTTIME,
  a.VALUE,
  a.VALUENUM,
  a.VALUEUOM,
  a.FLAG
FROM `physionet-data.mimiciii_clinical.labevents` AS a
INNER JOIN `physionet-data.mimiciii_clinical.d_labitems` AS b
ON a.itemid = b.itemid
INNER JOIN `physionet-data.mimiciii_clinical.icustays` as c
ON c.hadm_id = a.hadm_id
WHERE a.itemid in (50809)
AND a.hadm_id IS NOT NULL)
,

{query_all}
""")

In [0]:
df_lab_pco2 = run_query(f""" WITH HR as (
SELECT
  a.itemid,
  b.label,
  a.HADM_ID,
  c.icustay_id,
  a.CHARTTIME,
  a.VALUE,
  a.VALUENUM,
  a.VALUEUOM,
  a.FLAG
FROM `physionet-data.mimiciii_clinical.labevents` AS a
INNER JOIN `physionet-data.mimiciii_clinical.d_labitems` AS b
ON a.itemid = b.itemid
INNER JOIN `physionet-data.mimiciii_clinical.icustays` as c
ON c.hadm_id = a.hadm_id
WHERE a.itemid in (50818)
AND a.hadm_id IS NOT NULL)
,

{query_all}
""")

In [0]:
df_lab_creatinine = run_query(f""" WITH HR as (
SELECT
  a.itemid,
  b.label,
  a.HADM_ID,
  c.icustay_id,
  a.CHARTTIME,
  a.VALUE,
  a.VALUENUM,
  a.VALUEUOM,
  a.FLAG
FROM `physionet-data.mimiciii_clinical.labevents` AS a
INNER JOIN `physionet-data.mimiciii_clinical.d_labitems` AS b
ON a.itemid = b.itemid
INNER JOIN `physionet-data.mimiciii_clinical.icustays` as c
ON c.hadm_id = a.hadm_id
WHERE a.itemid in (50912)
AND a.hadm_id IS NOT NULL)
,

{query_all}
""")

In [0]:
df_lab_PT = run_query(f""" WITH HR as (
SELECT
  a.itemid,
  b.label,
  a.HADM_ID,
  c.icustay_id,
  a.CHARTTIME,
  a.VALUE,
  a.VALUENUM,
  a.VALUEUOM,
  a.FLAG
FROM `physionet-data.mimiciii_clinical.labevents` AS a
INNER JOIN `physionet-data.mimiciii_clinical.d_labitems` AS b
ON a.itemid = b.itemid
INNER JOIN `physionet-data.mimiciii_clinical.icustays` as c
ON c.hadm_id = a.hadm_id
WHERE a.itemid in (51274)
AND a.hadm_id IS NOT NULL)
,

{query_all}
""")

In [0]:
df_lab_lactate = run_query(f""" WITH HR as (
SELECT
  a.itemid,
  b.label,
  a.HADM_ID,
  c.icustay_id,
  a.CHARTTIME,
  a.VALUE,
  a.VALUENUM,
  a.VALUEUOM,
  a.FLAG
FROM `physionet-data.mimiciii_clinical.labevents` AS a
INNER JOIN `physionet-data.mimiciii_clinical.d_labitems` AS b
ON a.itemid = b.itemid
INNER JOIN `physionet-data.mimiciii_clinical.icustays` as c
ON c.hadm_id = a.hadm_id
WHERE a.itemid in (50813)
AND a.hadm_id IS NOT NULL)
,

{query_all}
""")

In [0]:
df_lab_INR = run_query(f""" WITH HR as (
SELECT
  a.itemid,
  b.label,
  a.HADM_ID,
  c.icustay_id,
  a.CHARTTIME,
  a.VALUE,
  a.VALUENUM,
  a.VALUEUOM,
  a.FLAG
FROM `physionet-data.mimiciii_clinical.labevents` AS a
INNER JOIN `physionet-data.mimiciii_clinical.d_labitems` AS b
ON a.itemid = b.itemid
INNER JOIN `physionet-data.mimiciii_clinical.icustays` as c
ON c.hadm_id = a.hadm_id
WHERE a.itemid in (51237)
AND a.hadm_id IS NOT NULL)
,

{query_all}
""")

In [0]:
df_lab_RBC['value'] = pd.to_numeric(df_lab_RBC['value'], errors='coerce')
df_lab_RBC = df_lab_RBC.groupby(by='icustay_id',as_index=False)['value'].mean()

In [0]:
df_lab_WBC = run_query(f""" WITH HR as (
SELECT
  a.itemid,
  b.label,
  a.HADM_ID,
  c.icustay_id,
  a.CHARTTIME,
  a.VALUE,
  a.VALUENUM,
  a.VALUEUOM,
  a.FLAG
FROM `physionet-data.mimiciii_clinical.labevents` AS a
INNER JOIN `physionet-data.mimiciii_clinical.d_labitems` AS b
ON a.itemid = b.itemid
INNER JOIN `physionet-data.mimiciii_clinical.icustays` as c
ON c.hadm_id = a.hadm_id
WHERE a.itemid in (51516)
AND a.hadm_id IS NOT NULL)
,

{query_all}
""")

In [0]:
df_lab_RBC = run_query(f""" WITH HR as (
SELECT
  a.itemid,
  b.label,
  a.HADM_ID,
  c.icustay_id,
  a.CHARTTIME,
  a.VALUE,
  a.VALUENUM,
  a.VALUEUOM,
  a.FLAG
FROM `physionet-data.mimiciii_clinical.labevents` AS a
INNER JOIN `physionet-data.mimiciii_clinical.d_labitems` AS b
ON a.itemid = b.itemid
INNER JOIN `physionet-data.mimiciii_clinical.icustays` as c
ON c.hadm_id = a.hadm_id
WHERE a.itemid in (51493)
AND a.hadm_id IS NOT NULL)
,

{query_all}
""")

## Create df_model

In [0]:
# Merge
print(df_bp_final.shape)
dfmodel = pd.merge(df_bp_final, df_features, left_on='id', right_on='icustay_id', how='left')
print(dfmodel.shape)

dfmodel = pd.merge(dfmodel, df_heartsingle, on='icustay_id', how='left')
dfmodel = dfmodel.rename(columns={'value':'heartrate'})
print(dfmodel.shape)

dfmodel = pd.merge(dfmodel, df_sat, on='icustay_id', how='left')
dfmodel = dfmodel.rename(columns={'value':'sat'})
print(dfmodel.shape)

dfmodel = pd.merge(dfmodel, df_vasopressor, on='icustay_id', how='left')
dfmodel = dfmodel.rename(columns={'value':'vasopressor'})
print(dfmodel.shape)

dfmodel = pd.merge(dfmodel, df_lab_ph, on='icustay_id', how='left')
dfmodel = dfmodel.rename(columns={'value':'ph'})
print(dfmodel.shape)

dfmodel = pd.merge(dfmodel, df_lab_hemaglobin, on='icustay_id', how='left')
dfmodel = dfmodel.rename(columns={'value':'hemaglobin'})
print(dfmodel.shape)

dfmodel = pd.merge(dfmodel, df_lab_crp, on='icustay_id', how='left')
dfmodel = dfmodel.rename(columns={'value':'crp'})
print(dfmodel.shape)

dfmodel = pd.merge(dfmodel, df_lab_po2, on='icustay_id', how='left')
dfmodel = dfmodel.rename(columns={'value':'po2'})
print(dfmodel.shape)

dfmodel = pd.merge(dfmodel, df_lab_base_excess, on='icustay_id', how='left')
dfmodel = dfmodel.rename(columns={'value':'base_excess'})
print(dfmodel.shape)

dfmodel = pd.merge(dfmodel, df_lab_glucose, on='icustay_id', how='left')
dfmodel = dfmodel.rename(columns={'value':'glucose'})
print(dfmodel.shape)

dfmodel = pd.merge(dfmodel, df_lab_pco2, on='icustay_id', how='left')
dfmodel = dfmodel.rename(columns={'value':'pco2'})
print(dfmodel.shape)

dfmodel = pd.merge(dfmodel, df_lab_creatinine, on='icustay_id', how='left')
dfmodel = dfmodel.rename(columns={'value':'creatinine'})
print(dfmodel.shape)

dfmodel = pd.merge(dfmodel, df_lab_PT, on='icustay_id', how='left')
dfmodel = dfmodel.rename(columns={'value':'PT'})
print(dfmodel.shape)

dfmodel = pd.merge(dfmodel, df_lab_lactate, on='icustay_id', how='left')
dfmodel = dfmodel.rename(columns={'value':'lactate'})
print(dfmodel.shape)

dfmodel = pd.merge(dfmodel, df_lab_INR, on='icustay_id', how='left')
dfmodel = dfmodel.rename(columns={'value':'INR'})
print(dfmodel.shape)

dfmodel = pd.merge(dfmodel, df_lab_WBC, on='icustay_id', how='left')
dfmodel = dfmodel.rename(columns={'value':'WBC'})

dfmodel = pd.merge(dfmodel, df_lab_RBC, on='icustay_id', how='left')
dfmodel = dfmodel.rename(columns={'value':'RBC'})

dfmodel.head(n=5)

(4923, 8)
(4923, 41)
(4923, 42)
(4923, 43)
(4923, 44)
(4923, 45)
(4923, 46)
(4923, 47)
(4923, 48)
(4923, 49)
(4923, 50)
(4923, 51)
(4923, 52)
(4923, 53)
(4923, 54)
(4923, 55)


,id,time_diff,invasive_value,ninv_value,invasive_time,ninv_time,femoral_yn,radial_yn,subject_id,hadm_id,...,po2,base_excess,glucose,pco2,creatinine,PT,lactate,INR,WBC,RBC
0,200033,0,54,59,2198-08-10 19:00:00,2198-08-10 19:00:00,0,1,56369.0,198650.0,...,172.0,9.0,81.0,35.0,0.5,14.6,1.3,1.3,NaN,0-2
1,200063,0,68,59,2141-03-13 09:00:00,2141-03-13 09:00:00,0,1,99052.0,129142.0,...,103.0,-4.0,106.0,39.0,0.8,16.1,1.5,1.4,2.0,1
2,200069,0,78,61,2195-12-12 10:00:00,2195-12-12 10:00:00,0,1,52323.0,122961.0,...,159.0,0.0,132.0,35.0,0.9,13.2,1.2,1.1,NaN,NaN
3,200075,0,77,54,2159-09-23 03:30:00,2159-09-23 03:30:00,0,1,67800.0,132255.0,...,114.0,-9.0,NaN,29.0,0.9,NaN,1.5,NaN,NaN,NaN
4,200141,3,56,51,2143-10-15 14:50:00,2143-10-15 14:53:00,0,1,93071.0,197473.0,...,84.0,-2.0,158.0,38.0,0.7,15.8,3.6,1.4,NaN,NaN


In [0]:
dfmodel.to_csv('gdrive/My Drive/Colab Notebooks/dfmodel_with_more_signals.csv', index=False)
# ! ls -ltr gdrive/'My Drive'/'Colab Notebooks'

total 21128
-rw------- 1 root root    18118 Feb  2 09:10 'Copy of MIMIC-III Datathon Tutorial'
drwx------ 2 root root     4096 Feb  2 12:13  pacmagic_feature_pacmed_analyzer
-rw------- 1 root root 18456782 Feb  2 15:12  dfmodel.csv
-rw------- 1 root root  1415403 Feb  3 09:33  dfmodel_with_signals.csv
-rw------- 1 root root   160312 Feb  3 10:06  Prepare_data_and_visualization.ipynb
-rw------- 1 root root  1578343 Feb  3 10:07  dfmodel_with_more_signals.csv


In [0]:
def get_last_measurement(df, df_signal, bp_date, signal_date):
  
  df_merge = pd.merge(
                    df, 
                    df_signal,
                    on = 'ICUSTAY_ID',
                    how = 'left'
  )
  
  df_merge = df_merge.loc[df_merge[bp_date] < df_merge[signal_date]]
  
  df_merge = df_merge.sort_values(['ICUSTAY_ID', bp_date, signal_date])
  
  df_merge = df_merge.drop_duplicates(subset = ['ICUSTAY_ID', bp_date], keep = 'last')
    
  return df_merge
